# Image Visualization

In [4]:
# Initialize gee engine
import ee
from ee import mapclient

try:
    ee.Initialize()
    print("Google earth engine initialize successful")
except ee.EEException:
    raise RuntimeError("Google earth engine initialize failed")

Google earth engine initialize successful


## RGB composites

In [ ]:
# Load an image
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Define the visualization prameters
vizParams  = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 0.5,
    'gamma': [0.95, 1.1, 1]
}

# Center the map and display the image
mapclient.centerMap(-122.1899, 37.5010, 10)
# using API mapclient.centerMap which different from js API: Map.setCenter
mapclient.addToMap(image, vizParams,'false color composite')

## Color palettes

In [ ]:
# Load an image
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Create an NDWI image, define visualization parameters and display
ndwi = image.normalizedDifference(['B3', 'B5'])
ndwiViz = {'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']}
mapclient.centerMap(-122.1899, 37.5010, 5)
mapclient.addToMap(ndwi, ndwiViz, 'NDWI')

## Masking

In [ ]:
# Mask the non-watery parts of the image, where NDWI < 0.4
ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))
mapclient.centerMap(-122.1899, 37.5010, 8)
mapclient.addToMap(ndwiMasked, ndwiViz, "NDWI masked")

## Visualization images

In [ ]:
# Create visualization layers.
# different from js api, visualization params can not write as a dict
imageRGB = image.visualize(bands=['B5', 'B4', 'B3'],max=0.5)
ndwiRGB = image.visualize(min=0.5, max=1, palette=['00FFFF', '0000FF'])

## Mosaicking

In [ ]:
# Mosaic the visualization layers and display (or export).
mosaic = ee.ImageCollection([imageRGB, ndwiRGB]).mosaic()
mapclient.centerMap(-122.1899, 37.5010, 10)
mapclient.addToMap(mosaic, {}, 'mosaic')

## Clipping

In [ ]:
# Create a circle by drawing a 20000 meter buffer around a point
roi = ee.Geometry.Point([-122.4481, 37.7599]).buffer(20000)
mapclient.centerMap(-122.1899, 37.5010, 10)
mapclient.addToMap(mosaic.clip(roi), {}, 'mosaic')

## Rendering categorical maps

In [ ]:
# Load 2012 MODIS land cover and select the IGBP classification
cover = ee.Image('MODIS/051/MCD12Q1/2012_01_01').select('Land_Cover_Type_1')

# Define a palette for the 18 distinct land cover classes
igbpPalette = [
  'aec3d4', # water
  '152106', '225129', '369b47', '30eb5b', '387242', # forest
  '6a2325', 'c3aa69', 'b76031', 'd9903d', '91af40',  # shrub, grass
  '111149', # wetlands
  'cdb33b', # croplands
  'cc0013', # urban
  '33280d', # crop mosaic
  'd7cdcc', # snow and ice
  'f7e084', # barren
  '6f6f6f'  # tundra
]

# Specify the min and max labels and the color palette matching the labels
cover.visualize(min=0, max=17, palette=igbpPalette)
mapclient.centerMap(-99.229, 40.413, 5)
mapclient.addToMap(cover, {}, "IGBP classification")